# Showcases. HW18

In [27]:
from custom_random_forest import RandomForestClassifierCustom
from sklearn.datasets import make_classification

from bioinf_tools import AminoAcidSequence, DNASequence, RNASequence, run_genscan
from bio_files_processor import OpenFasta

## BioInf tools

In [28]:
aa = AminoAcidSequence('HKALAY')
assert aa.check_alphabet() # Проверяем корректность аминокислоты
print(aa, aa.define_charge()) # __repr__ и функция оценки заряда аминокислот

HKALAY {'Positive': 2, 'Negative': 0, 'Neutral': 4}


In [29]:
dna = DNASequence('ATGC')
assert dna.check_alphabet() # Проверяем корректность днк
print(f'{dna} - DNA sequence') # __repr__
print('Transcribed:', dna.transcribe(), isinstance(dna.transcribe(), RNASequence)) # транскрипция и проверка изменения класса на RNASequence
print('Complementary strand:', dna.complement(), isinstance(dna.complement(), DNASequence)) # репликация и проверка возвращения того же класса

ATGC - DNA sequence
Transcribed: AUGC True
Complementary strand: TACG True


In [30]:
rna = RNASequence('AUGC')
assert rna.check_alphabet() # Проверяем корректность рнк
print(f'{rna} - RNA sequence') # __repr__
print('Complementary rna:', rna.complement(), isinstance(rna.complement(), RNASequence)) 
# транскрипт с комплементарной цепи соответствующей днк и проверка возвращения того же класса

AUGC - RNA sequence
Complementary rna: UACG True


In [31]:
results = run_genscan(sequence_file='data/NUDT9_gene_multiline.fasta')
results # в результате работы генскан возвращает статус объект GenscanOutput, хранящий код request запроса, предсказанные cds, интроны и экзоны

GenscanOutput(status=200, cds_list=[{'index': 1, 'name': '/tmp/05_02_24-17:25:00.fasta|GENSCAN_predicted_peptide_1|351_aa', 'sequence': 'MAGRLLGKALAAVSLSLALASVTIRSSRCRGIQAFRNSFSSSWFHLNTNVMSGSNGSKENSHNKARTSPYPGSKVERSQVPNEKVGWLVEWQDYKPVEYTAVSVLAGPRWADPQISESNFSPKFNEKDGHVERKSKNGLYEIENGRPRNPAGRTGLVGRGLLGRWGPNHAADPIITSMHFMIQNSVVQDFINFFRSSGAGVKFSDSQASLLEIVQGMVDPGEKISATLKREFGEEALNSLQKTSAEKREIEEKLHKLFSQDHLVIYKGYVDDPRNTDNAWMETEAVNYHDETGEIMDNLMLEAGDDAGKVKWVDINDKLKLYASHSQFIKLVAEKRDAHWSEDSEADCHAL'}, {'index': 2, 'name': '/tmp/05_02_24-17:25:00.fasta|GENSCAN_predicted_peptide_2|115_aa', 'sequence': 'MIKILEEHMLTKFHDTGFGDDFLDMTPKVQATREKIAPSPGRGYLMGTRTCLPLGSRTGVAVPIYKSQIKSPPTIRVHRFPQCIEKNPDQHVTPKPPAPAAWGNRGHEPRNFLSA'}, {'index': 3, 'name': '/tmp/05_02_24-17:25:00.fasta|GENSCAN_predicted_peptide_3|165_aa', 'sequence': 'MSGSNGSKENSHNKARTSPYPGSKVERSQVPNEKVGWLVEWQDYKPVEYTAVSVLAGPRWADPQISESNFSPKFNEKDGHVERKSKNGLYEIENGRPRNPAGRTGLVGRGLLGRWGPNHAADPIITSMHFMIQNSVVQDFINFFRSSGAGVKFSDSQASLLEIVQ'}], intron_list=[[{'i

## Bio files processor

In [32]:
with OpenFasta('data/NUDT9_gene_oneline.fasta') as f:
    print(f.read_record()) # возвращает объект FastaRecord, хранящий id, описание последовательности и саму последовательность, аналог readline

with OpenFasta('data/NUDT9_gene_oneline.fasta') as f:
    print(f.read_records()) # возвращает список со всеми последовательностями в формате FastaRecord, которые можно найти в файле, аналог readlines

FastaRecord(id=NC_000004.12:87422573-87459455, description=NUDT9 [organism=Homo sapiens] [GeneID=53343] [chromosome=4], sequence=...)
[FastaRecord(id=NC_000004.12:87422573-87459455, description=NUDT9 [organism=Homo sapiens] [GeneID=53343] [chromosome=4], sequence=...), FastaRecord(id=NC_060928.1:90749045-90788493, description=NUDT9 [organism=Homo sapiens] [GeneID=53343] [chromosome=4], sequence=...)]


In [33]:
# Если файл содержит multiline fasta выдаст специальную ошибку
with OpenFasta('data/NUDT9_gene_multiline.fasta'):
    pass

FastaFormatError: Check fasta format, it should be single line.
You can change multiline to single line fasta with "translate_multiline_to_singleline" function!

## Рандомный лес, поддерживающий параллельные выисления

In [34]:
X, y = make_classification(n_samples=100000)
random_forest = RandomForestClassifierCustom(max_depth=30, n_estimators=10, 
                                             max_features=2, random_state=42)

In [41]:
%%time
random_forest.fit(X, y, n_jobs=1) 
# решил не делить на train и test, так как цель оценить время выполнения -> лучше, чтобы нагрузка на функцию была максимальной 

CPU times: user 266 ms, sys: 228 ms, total: 494 ms
Wall time: 14.7 s


RandomForestClassifierCustom(max_depth=30, max_features=2, random_state=42)

In [49]:
%%time
r1 = random_forest.predict(X, n_jobs=1)

CPU times: user 421 ms, sys: 205 ms, total: 626 ms
Wall time: 1.05 s


In [40]:
%%time
random_forest.fit(X, y, n_jobs=2)

CPU times: user 225 ms, sys: 240 ms, total: 465 ms
Wall time: 10.5 s


RandomForestClassifierCustom(max_depth=30, max_features=2, random_state=42)

In [52]:
%%time
r2 = random_forest.predict(X, n_jobs=2)

CPU times: user 168 ms, sys: 167 ms, total: 334 ms
Wall time: 565 ms


In [39]:
assert all(r1 == r2) == True

Как мы видим, результаты предсказаний не изменяются, а выигрыш во времени >= 1.5 раза!

| n_jobs        | 1               | 2                |
|---------------|-----------------|------------------|
| fit, wall time     | 14.7 s | 10.5 s  |
| predict, wall time | 1.05 s  | 565 ms |